<a href="https://colab.research.google.com/github/dimna21/ML_Assignment4/blob/main/FER2013.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb

In [9]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dimna21 (dimna21-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# 1) Load the CSV
csv_path = "/content/drive/MyDrive/FER_data/fer2013/fer2013.csv"
df = pd.read_csv(csv_path)

# 2) Split by Usage
df_train = df[df['Usage']=="Training"].copy()
df_val   = df[df['Usage']=="PublicTest"].copy()
df_test  = df[df['Usage']=="PrivateTest"].copy()

In [4]:
# 3) Balance function: upsample & random ±10 intensity shifts
def balance_dataset(df, target_count, img_shape=(48,48)):
    def augment(pix_str):
        arr = np.fromstring(pix_str, sep=' ', dtype=int).reshape(img_shape)
        shift = np.random.randint(-10, 11)
        arr = np.clip(arr + shift, 0, 255).astype(int)
        return ' '.join(map(str, arr.ravel()))
    parts = [df]
    for emo, grp in df.groupby('emotion'):
        n = len(grp)
        if n < target_count:
            extra = grp.sample(n=target_count-n, replace=True).copy()
            extra['pixels'] = extra['pixels'].map(augment)
            parts.append(extra)
    return pd.concat(parts, ignore_index=True)

max_count = df_train['emotion'].value_counts().max()
balanced_train = balance_dataset(df_train, target_count=max_count)

In [5]:
# 4) Dataset class
class FERDataset(Dataset):
    def __init__(self, dataframe):
        self.pixels = dataframe['pixels'].values
        self.labels = dataframe['emotion'].values.astype(int)
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        arr = np.fromstring(self.pixels[idx], sep=' ', dtype=np.uint8).reshape(48,48)
        arr = arr.astype(np.float32) / 255.0
        tensor = torch.from_numpy(arr).unsqueeze(0)  # shape [1,48,48]
        return tensor, self.labels[idx]


In [6]:
# 5) Simple conv-based net: 5×Conv3×3 → Dropout → FC → FC
class BaselineModel(nn.Module):
    def __init__(self,
                 in_channels: int = 1,
                 conv_channels: int = 32,
                 hidden_dim: int    = 256,
                 num_classes: int   = 7,
                 dropout_p: float   = 0.5):
        super().__init__()
        self.convs = nn.Sequential(
            nn.Conv2d(in_channels,   conv_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(conv_channels, conv_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(conv_channels, conv_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )
        self.dropout = nn.Dropout(p=dropout_p)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(conv_channels * 48 * 48, hidden_dim)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.convs(x)          # [B, conv_channels, 48, 48]
        x = self.dropout(x)        # dropout on feature maps
        x = self.flatten(x)        # [B, conv_channels*48*48]
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)            # [B, num_classes]
        return x

# Example instantiation
model = BaselineModel()
print(model)

BaselineModel(
  (convs): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=73728, out_features=256, bias=True)
  (relu): ReLU(inplace=True)
  (fc2): Linear(in_features=256, out_features=7, bias=True)
)


In [7]:
# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [8]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# DataLoaders
batch_size = 128
train_ds = FERDataset(balanced_train)
val_ds   = FERDataset(df_val)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=2)
val_dl   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=2)


In [10]:
import wandb
import torch
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

def train_model(
    model,
    train_loader,
    val_loader,
    test_loader,           # new: DataLoader for your held-out test set
    criterion,
    optimizer,
    device,
    epochs=5,
    class_names=None      # list of names for your classes
):
    wandb.init(
        project="ML_Assignment4",
        config={
            "epochs": epochs,
            "batch_size": train_loader.batch_size,
            "optimizer": optimizer.__class__.__name__,
            "lr": optimizer.param_groups[0]["lr"],
            "criterion": criterion.__class__.__name__,
        }
    )
    cfg = wandb.config
    wandb.watch(model, log="all", log_freq=100)

    history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}
    model.to(device)

    for epoch in range(1, cfg.epochs + 1):
        # — TRAIN —
        model.train()
        tloss = 0.0
        tcorrect = 0
        for X, y in tqdm(train_loader, desc=f"[Train] epoch {epoch}"):
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(X)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()
            tloss += loss.item() * X.size(0)
            tcorrect += (logits.argmax(1) == y).sum().item()
        train_loss = tloss / len(train_loader.dataset)
        train_acc  = tcorrect / len(train_loader.dataset)
        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)

        # — VALIDATE —
        model.eval()
        vloss = 0.0
        vcorrect = 0
        with torch.no_grad():
            for X, y in tqdm(val_loader, desc=f"[Val] epoch {epoch}"):
                X, y = X.to(device), y.to(device)
                logits = model(X)
                loss = criterion(logits, y)
                vloss += loss.item() * X.size(0)
                vcorrect += (logits.argmax(1) == y).sum().item()
        val_loss = vloss / len(val_loader.dataset)
        val_acc  = vcorrect / len(val_loader.dataset)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        print(f"Epoch {epoch}/{cfg.epochs} — "
              f"Train loss {train_loss:.4f}, acc {train_acc:.4f} | "
              f"Val loss {val_loss:.4f}, acc {val_acc:.4f}")

        wandb.log({
            "epoch": epoch,
            "train/loss": train_loss,
            "train/accuracy": train_acc,
            "val/loss": val_loss,
            "val/accuracy": val_acc,
        })

    # — FINAL TEST EVAL —
    if test_loader is not None:
        model.eval()
        preds, targets = [], []
        with torch.no_grad():
            for X, y in tqdm(test_loader, desc="[Test]"):
                X = X.to(device)
                logits = model(X)
                preds.extend(logits.argmax(1).cpu().tolist())
                targets.extend(y.tolist())

        test_acc = accuracy_score(targets, preds)
        f1s = f1_score(targets, preds, average=None)
        cm = confusion_matrix(targets, preds)

        # log test accuracy
        wandb.log({"test/accuracy": test_acc})

        # log per-class F1
        for idx, cls in enumerate(class_names or map(str, range(len(f1s)))):
            wandb.log({f"test/f1_{cls}": f1s[idx]})

        # log confusion matrix
        wandb.log({
            "test/confusion_matrix": wandb.plot.confusion_matrix(
                probs=None,
                y_true=targets,
                preds=preds,
                class_names=list(class_names or map(str, range(len(f1s))))
            )
        })

        print(f"Test Acc: {test_acc:.4f}")
        print("Test F1 per class:", dict(zip(class_names or range(len(f1s)), f1s)))
        print("Confusion matrix:\n", cm)

    wandb.finish()
    return history


In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

batch_size = 128
train_ds = FERDataset(balanced_train)
val_ds   = FERDataset(df_val)
test_ds  = FERDataset(df_test)

train_dl = DataLoader(train_ds,  batch_size=batch_size, shuffle=True,  num_workers=2)
val_dl   = DataLoader(val_ds,    batch_size=batch_size, shuffle=False, num_workers=2)
test_dl  = DataLoader(test_ds,   batch_size=batch_size, shuffle=False, num_workers=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model  = BaselineModel().to(device)

history = train_model(
    model=model,
    train_loader=train_dl,
    val_loader=val_dl,
    test_loader=test_dl,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    epochs=5,
    class_names=['surprise','fear','disgust','happy','sad','angry','neutral']
)

[Val] epoch 1: 100%|██████████| 29/29 [00:01<00:00, 28.79it/s]


Epoch 1/5 — Train loss 1.9468, acc 0.1425 | Val loss 1.9386, acc 0.1819


[Val] epoch 2: 100%|██████████| 29/29 [00:00<00:00, 57.17it/s]


Epoch 2/5 — Train loss 1.9467, acc 0.1434 | Val loss 1.9386, acc 0.1819


[Val] epoch 3: 100%|██████████| 29/29 [00:00<00:00, 56.02it/s]


...

[Val] epoch 5: 100%|██████████| 29/29 [00:01<00:00, 28.62it/s]


Epoch 5/5 — Train loss 1.9468, acc 0.1432 | Val loss 1.9386, acc 0.1819


[Test]: 100%|██████████| 29/29 [00:00<00:00, 59.25it/s]


Test Acc: 0.1655
Test F1 per class: {'surprise': np.float64(0.0), 'fear': np.float64(0.0), 'disgust': np.float64(0.0), 'happy': np.float64(0.0), 'sad': np.float64(0.284006693760459), 'angry': np.float64(0.0), 'neutral': np.float64(0.0)}
Confusion matrix:
 [[  0   0   0   0 491   0   0]
 [  0   0   0   0  55   0   0]
 [  0   0   0   0 528   0   0]
 [  0   0   0   0 879   0   0]
 [  0   0   0   0 594   0   0]
 [  0   0   0   0 416   0   0]
 [  0   0   0   0 626   0   0]]


epoch,▁▃▅▆█
test/accuracy,▁
test/f1_angry,▁
test/f1_disgust,▁
test/f1_fear,▁
test/f1_happy,▁
test/f1_neutral,▁
test/f1_sad,▁
test/f1_surprise,▁
train/accuracy,▁█▅▃▆
train/loss,▇▁█▄▆
